# SBML_to_ODE to odeint
In this notebook, we will wrangle the SBML_to_ODE function outputs so they can be compatible with scipy.odeint().<br>
7.30.2020<br>
---

In [32]:
# Imports
from sympy.utilities.lambdify import lambdify
from sympy import Symbol

from scipy.integrate import odeint

import bokeh.io
import bokeh.plotting
bokeh.io.output_notebook()

import colorcet

Loading BokehJS ...

Load the sbml_to_ode2 func:

In [1]:
%run sbml_to_ode2.py

Use the function with .xml file:

In [2]:
filename = 'CRN_GOOD.xml'
x,f,P,params_values,x_init = sbml_to_ode2(filename)

Now, let's make it compatible with scipy.odeint()

In [3]:
# Parameters
param_eval_f = [None]*len(f)
i = 0
while i < len(f):
    param_eval_f[i] = f[i].subs(dict(zip(P,params_values)))
    i += 1

In [12]:
# Lamdbify 
t = Symbol('t')
func = lambdify([t, x, P], f)
func_p = lambdify([t, x], param_eval_f)

In [21]:
# Set variables
n = len(x)
timepoints = np.linspace(0,75,100)

In [22]:
# Solve
solutions = odeint(lambda t, x:func_p(t,x), x_init, timepoints, tfirst = True)
#solutions = np.reshape(solutions, len(timepoints), n)

In [28]:
# Make sure this is num species
len(solutions[0])

68

Let's take a look at plot of atp, glucose, isobutanol to make sure it works as expected.

In [33]:
p = bokeh.plotting.figure(width = 450, height = 300,
                         title = 'Test sbml to ode code',
                         x_axis_label = 'time (hrs)',
                         y_axis_label = 'concentration (mM)')

palette = colorcet.b_glasbey_category10
p.line(timepoints, solutions[:,1], legend_label = 'atp', line_width = 2, color = palette[0])
p.line(timepoints, solutions[:,2], legend_label = 'glucose', line_width = 2, color = palette[1])
p.line(timepoints, solutions[:,62], legend_label = 'isobutanol', line_width = 2, color = palette[2])


bokeh.io.show(p)

In [34]:
#watermark
%reload_ext watermark
%watermark -v -p sympy,scipy,bokeh,colorcet,jupyterlab

CPython 3.7.7
IPython 7.13.0

sympy 1.5.1
scipy 1.4.1
bokeh 2.0.2
colorcet 2.0.2
jupyterlab 1.2.6
